In [3]:
%run 2.Ecoli_model.ipynb

Putrescine
ptrc_p
Putrescine
ptrc_c
Putrescine
ptrc_e
41.09128205128201
9.56477064220184
N-Acetyl-L-glutamate 5-semialdehyde
acg5sa_c
N2-Succinyl-L-glutamate 5-semialdehyde
sucgsa_c
L-Glutamate 5-semialdehyde
glu5sa_c
Ornithine
orn_e
Ornithine
orn_p
Ornithine
orn_c
2-Oxoglutarate
akg_e
2-Oxoglutarate
akg_c
2-Oxoglutarate
akg_p
L-Glutamate 1-semialdehyde
glu1sa_c
L-Glutamate
glu__L_c
L-Glutamate 5-semialdehyde
glu5sa_c
L-Glutamate
glu__L_e
L-Glutamate
glu__L_p
L-Glutamate 5-phosphate
glu5p_c
glu5sa_c + glu__L_c --> akg_c + orn_c
53.02333333333331
45.787428571428464
9.92914285714285
L-Arginine
arg__L_c
L-Arginine
arg__L_e
L-Arginine
arg__L_p
Urea CH4N2O
urea_c
Urea CH4N2O
urea_p
Urea CH4N2O
urea_e
H2O H2O
h2o_p
H2O H2O
h2o_c
H2O H2O
h2o_e
arg__L_c + h2o_c --> orn_c + urea_c
45.787428571428464
9.929142857142857


# Gene knock out strategies

In [4]:
#We would like to utilize the OptKnock built in algoritm in COBRApy
#First we make a list of essential genes, 
#second we make a list of non-essential genes

In [5]:
import cobra

In [6]:
def get_blocked_reactions(model):
    fva_res = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0)
    blocked = [
        model.reactions.get_by_id(reaction) for reaction, row in fva_res.iterrows()
        if (round(row["minimum"], 5) == round(row["maximum"], 5) == 0)
    ]
    return blocked

In [7]:
#def get_blocked_reactions(model):
  #  fva_res = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0)
  #  blocked = [
  #      model.reactions.get_by_id(reaction) for reaction, row in fva_res.data_frame.iterrows()
  #      if (round(row["lower_bound"], 5) == round(row["upper_bound"], 5) == 0)
  #  ]
  #  return blocked

In [8]:
blockedreactions = get_blocked_reactions(model)

In [11]:
essentiality = {}
importantgenes = {}
targetgenes = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality[gene] = model.slim_optimize(error_value=0.)
        #print(model.slim_optimize(error_value=0.))
        if model.slim_optimize(error_value=0.) < 0.5:
            importantgenes[gene] = model.slim_optimize(error_value=0.)
        else:
            targetgenes[gene] = model.slim_optimize(error_value=0.)
            

In [12]:
#essentiality

In [14]:
#importantgenes

In [ ]:
#targetgenes

In [ ]:
#Make loop of our targetgenes list to append all reactions per gene

In [ ]:
#targetgenes.index[0]

In [15]:
#model.genes.b2551.reactions
importantreactions = []
for gene in importantgenes:
    for reaction in gene.reactions:
        importantreactions.append(reaction)
importantreactions = list(set(importantreactions))


In [16]:
len(importantreactions)

418

In [17]:
l3 = [x.id for x in importantreactions if x not in blockedreactions]

In [19]:
len(l3)

383

In [ ]:
# l5 = set(importantreactions).intersection(blockedreactions)

In [ ]:
#Then we make a list of reactions we can knock out based on the knockable genes

In [ ]:
#Then we run the algoritm for OptKnock

In [20]:
import cameo

In [21]:
from cameo.strain_design.deterministic.linear_programming import OptKnock

In [22]:
#for reaction in model.reactions.query('ATP', 'name'):
    #print(reaction.name)
    #print(reaction.id)

In [23]:
for reaction in model.reactions.query('CO2', 'name'):
    print(reaction.name)
    print(reaction.id)

CO2 exchange
EX_co2_e
CO2 transporter via diffusion (periplasm)
CO2tpp
CO2 transport via diffusion (extracellular to periplasm)
CO2tex


In [24]:
#for reaction in model.reactions.query('CO', 'name'):
  #  print(reaction.name)
   # print(reaction.id)

In [25]:
optknock = OptKnock(model, fraction_of_optimum=0.1, exclude_reactions= l3)

In [26]:
optknock

In [ ]:
#result = optknock.run(max_knockouts=3, target="EX_ptrc_e", biomass="BIOMASS_Ec_iML1515_core_75p37M")